# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770403740140                   -0.52    9.0         
  2   -2.771687450037       -2.89       -1.32    1.0    112ms
  3   -2.771714180343       -4.57       -2.49    1.0    124ms
  4   -2.771714653171       -6.33       -3.20    2.0    130ms
  5   -2.771714710253       -7.24       -3.67    1.0   93.9ms
  6   -2.771714714697       -8.35       -4.38    1.0    113ms
  7   -2.771714715249       -9.26       -5.38    2.0    111ms
  8   -2.771714715249      -12.68       -5.89    2.0    126ms
  9   -2.771714715250      -12.55       -6.51    1.0    108ms
 10   -2.771714715250      -13.78       -7.44    2.0    120ms
 11   -2.771714715250      -14.05       -8.88    2.0    126ms


-0.0001345736945164098

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770446838484                   -0.53    9.0         
  2   -2.771773735413       -2.88       -1.30    1.0   91.6ms
  3   -2.771801538664       -4.56       -2.62    1.0    132ms
  4   -2.771802070609       -6.27       -3.92    2.0    108ms
  5   -2.771802073956       -8.48       -4.19    2.0    140ms
  6   -2.771802074451       -9.31       -5.01    1.0   96.0ms
  7   -2.771802074476      -10.61       -5.75    2.0    113ms
  8   -2.771802074476      -12.46       -6.38    1.0    114ms
  9   -2.771802074476      -13.29       -7.07    2.0    123ms
 10   -2.771802074476   +  -13.89       -7.95    1.0    117ms
 11   -2.771802074476      -14.07       -8.86    2.0    117ms


0.017612221496947928

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.0001345736945164098
Displaced dipole:  0.017612221496947928
Polarizability :   1.7746795191464337


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920647691e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551173497e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852765770979e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694593875386e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787627499e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.274979524117e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 7.157361487864e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 7.157361487864e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.711588924131e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.568104755867e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 4.958499280133e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 6.244476122654e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 6.244476122654e-14
┌ Info: GMRES linsolve converged at iteration 2, step

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.